In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [2]:
import os
path = '/content/gdrive/MyDrive/MachineVision/Re1'

In [3]:
import numpy as np
import cv2 as cv
import glob
from google.colab.patches import cv_imshow


criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER,30,0.001)

objp = np.zeros((6*8,3),np.float32) #초기화
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2) #
objpoints = [] 
imgpoints = [] 
images = glob.glob('/content/gdrive/MyDrive/MachineVision/Re1/re*.jpg') #-영상 저장해둔곳

In [ ]:
for fname in images:
  img = cv.imread(fname) #fname 경로에서 imread
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  #Find the chess board corners 모서리찾기
  ret, corners = cv.findChessboardCorners(gray,(8,6),None) #그래이 스케일의 7,6짜리 체스보드를 검출하라는 뜻
  #If found, add object points, image points(after refining them)
  if ret == True:
    objpoints.append(objp)
    corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria) #보다더 정확하게 코너의 위치를 보정
    imgpoints.append(corners)
    #Draw and display the corners
    cv.drawChessboardCorners(img,(8,6),corners2,ret)
    cv_imshow(img)
    

In [5]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None, None) 

In [6]:
mtx

array([[1.09201968e+03, 0.00000000e+00, 5.38906437e+02],
       [0.00000000e+00, 1.08515707e+03, 7.27848806e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [7]:
rvecs

[array([[-0.50039384],
        [ 0.20097509],
        [ 1.4328519 ]]), array([[-0.22407509],
        [-0.28429392],
        [ 1.5814638 ]]), array([[-0.0508498 ],
        [-0.3539706 ],
        [ 0.40739027]]), array([[ 0.15833766],
        [-0.29627395],
        [-0.93519908]]), array([[-0.23592123],
        [-0.27490249],
        [ 1.54005891]]), array([[-0.35464056],
        [ 0.58570486],
        [ 1.54734385]])]

In [8]:
tvecs

[array([[ 1.65548676],
        [-4.43443485],
        [15.49924628]]), array([[ 1.8523695 ],
        [-3.64823396],
        [15.81196613]]), array([[-2.04795061],
        [-4.02326226],
        [15.29765818]]), array([[-3.98954388],
        [ 0.46082311],
        [14.52026324]]), array([[ 1.46648504],
        [-3.84960665],
        [16.75888695]]), array([[ 1.34619019],
        [-2.78011643],
        [14.99736111]])]

In [11]:
def draw(img,corners,imgpts):
  imgpts= np.int32(imgpts).reshape(-1,2)
  #draw ground floor in green
  img = cv.drawContours(img,[imgpts[:4]],-1,(0,255,0),-3)
  #draw pillars in blue color
  for i,j in zip(range(5),range(4,8)):
    img = cv.line(img,tuple(imgpts[i]),tuple(imgpts[j]),(255,0,0 ),3)
  #draw top layer in red color
  img = cv.drawContours(img,[imgpts[4:]],-1,(0,0,255),3)
  return img

In [12]:
axis = np.float32([[0,0,0],[0,3,0],[3,3,0],[3,0,0],[0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3]])

i=0
for fname in glob.glob('/content/gdrive/MyDrive/MachineVision/Re1/re*.jpg'):
  img = cv.imread(fname)
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  ret, corners = cv.findChessboardCorners(gray,(8,6),None)
  if ret == True:
    #project 3D points to image plane
    imgpts, jac = cv.projectPoints(axis, rvecs[i],tvecs[i],mtx, dist)
    i=i+1
    img= draw(img,corners2, imgpts)
    cv_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
axis = np.float32([[0,0,0],[0,3,0],[1,3,0],[1,0,0],[0,0,-5],[0,3,-5],[1,3,-5],[1,0,-5]])

i=0
for fname in glob.glob('/content/gdrive/MyDrive/MachineVision/Re1/re*.jpg'):
  img = cv.imread(fname)
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  ret, corners = cv.findChessboardCorners(gray,(8,6),None)
  if ret == True:
    #project 3D points to image plane
    imgpts, jac = cv.projectPoints(axis, rvecs[i],tvecs[i],mtx, dist)
    i=i+1
    img= draw(img,corners2, imgpts)
    cv_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
axis2 = np.float32([[2,2,0],[2,5,0],[3,5,0],[3,2,0],[2,2,-5],[2,5,-5],[3,5,-5],[3,2,-5]])

i=0
for fname in glob.glob('/content/gdrive/MyDrive/MachineVision/Re1/re*.jpg'):
  img = cv.imread(fname)
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  ret, corners = cv.findChessboardCorners(gray,(8,6),None)
  if ret == True:
    #project 3D points to image plane
    imgpts, jac = cv.projectPoints(axis2, rvecs[i],tvecs[i],mtx, dist)
    i=i+1
    img= draw(img,corners2, imgpts)
    cv_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [24]:
axis = np.float32([[0,0,0],[0,3,0],[1,3,0],[1,0,0],[0,0,-5],[0,3,-5],[1,3,-5],[1,0,-5]])
axis2 = np.float32([[2,2,0],[2,5,0],[3,5,0],[3,2,0],[2,2,-5],[2,5,-5],[3,5,-5],[3,2,-5]])

i=0
for fname in glob.glob('/content/gdrive/MyDrive/MachineVision/Re1/re*.jpg'):
  img = cv.imread(fname)
  gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  ret, corners = cv.findChessboardCorners(gray,(8,6),None)
  if ret == True:
    #project 3D points to image plane
    imgpts, jac = cv.projectPoints(axis, rvecs[i],tvecs[i],mtx, dist)
    img= draw(img,corners2, imgpts)
    imgpts, jac = cv.projectPoints(axis2, rvecs[i],tvecs[i],mtx, dist)
    img= draw(img,corners2, imgpts)
    i=i+1
    cv_imshow(img)

Output hidden; open in https://colab.research.google.com to view.